## Evaluation of pipelines via EvalAp

In [11]:
import requests
from dotenv import load_dotenv
import os
import time

# Load environment variables from .env file
load_dotenv()


OPENWEBUI_TOKEN = os.getenv('OPENWEBUI_TOKEN')

def chat_with_model(model, prompt):
    url = 'http://localhost:8080/api/chat/completions'
    headers = {
        'Authorization': f'Bearer {OPENWEBUI_TOKEN}',
        'Content-Type': 'application/json'
    }
    data = {
      "model": model,
      "messages": [
        {
          "role": "user",
          "content": prompt
        }
      ]
    }
    response = requests.post(url, headers=headers, json=data, timeout=10000)
    #response = requests.get(url, headers=headers)

    return response



def chat_with_model_streaming(model, prompt):
    url = 'http://localhost:8080/api/chat/completions'
    headers = {
        'Authorization': f'Bearer {OPENWEBUI_TOKEN}',
        'Content-Type': 'application/json'
    }
    data = {
      "model": model,
      "messages": [
        {
          "role": "user",
          "content": prompt
        }
      ],
      "stream": True
    }

    start_time = time.time()
    response = requests.post(url, headers=headers, json=data, stream=True, timeout=10000)
    #response = requests.get(url, headers=headers)

    full_content = ""
    for line in response.iter_lines():
        if line:
            decoded_line = line.decode('utf-8')
            if decoded_line.startswith('data: '):
                data_content = decoded_line[6:]  # Remove 'data: ' prefix
                try:
                    import json
                    chunk_data = json.loads(data_content)
                    if chunk_data.get("event", {}).get("data", {}).get("content"):
                        full_content += chunk_data["event"]["data"]["content"]
                except:
                    pass
    end_time = time.time()
    print(f"Time taken: {end_time - start_time} seconds")
    return full_content, end_time - start_time


In [ ]:
model = "assistant-service-public--recherche-approfondie" #"assistant-service-public"
prompt = "Un contrat de CDI doit-il être écrit ?"

answer, time_spent = chat_with_model_streaming(model, prompt)
print(answer)
print(time_spent)

In [ ]:
import pandas as pd

EG1_API_URL = os.getenv('EG1_API_URL')
headers = {
    'Authorization': f'Bearer {os.getenv('EG1_API_KEY')}',
    'Content-Type': 'application/json'
}

response = requests.get(f'{EG1_API_URL}/dataset/1?with_df=true', headers=headers)
df = pd.read_json(response.json()['df'])
df.head()

In [ ]:
answers = []
times = []
for index, row in df.iterrows():
    print(f"======================= {index} ========================")
    prompt = row['query']
    answer, time_spent = chat_with_model_streaming(model, prompt)
    answers.append(answer)
    times.append(time_spent)
    print(f"prompt: {prompt}")
    print(f"answer: {answer}")
    #print(f"time_spent: {time_spent}")
    print(f"===================== end {index} ======================")
df['output'] = answers
df['generation_time'] = times
df.head()


In [ ]:
## Push exp 

headers = {"Authorization": f"Bearer {os.getenv('EG1_API_KEY')}", "Accept-Encoding": "gzip"}

metrics = ["judge_precision", "judge_notator", "judge_exactness", "output_length", "answer_relevancy"]

# Lauching the experiments
experiment_ids = []
experiment = {
            "experiment_set_id" : "54",
            "name" : model + "- Pipeline V1",
            "readme" : model + "- Pipeline V1",
            "model":{"aliased_name" : model + "- Pipeline V1", "output":df["output"].values.tolist(), "execution_time":df["generation_time"].astype(int).values.tolist()},
            "dataset" : "MFS_questions_v01",
            "metrics": metrics,
            "judge_model":"gpt-4o",
            }

response = requests.post(f'{EG1_API_URL}/experiment', json=experiment, headers=headers)
resp = response.json()

if "id" in resp:
    experiment_ids.append(resp["id"])
    print(f'Created experiment: {resp["name"]} ({resp["id"]}), status: {resp["experiment_status"]}')
else:
    print(resp)

In [ ]:
df.to_csv(f"results/{model}.csv", index=False)